# Финальный проект

Финальный проект на курсе по машинному обучению на курсере

В этом ноутбуке предлагается решать задачу предсказания средней цены квадратного метра жилья в некоторых районах Москвы и Московской области. В качестве метрики будем использовать MAPE.

MAPE (Mean Absolute Percentage Error) = $\dfrac{1}{\ell}\sum\limits_{i=1}^{\ell} \left| \dfrac{y_i - a(x_i)}{y_i} \right|$

## Импорт библиотек и чтение данных

In [3]:
import pandas as pd
import numpy as np
# !pip uninstall -y seaborn
# !pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid', rc={'figure.figsize': (16, 8)})

pd.options.display.max_columns = 100

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

housebase = pd.read_csv('housebase.csv')

### train.csv и test.csv

Описание полей в файлах train.csv и test.csv:

**month** - месяц, за который указаны данные в датасете

**city_quadkey** - полигон, его координаты можно его узнать из скрипта quadkey.py

**apartment_ratio** - доля квартир в этом районе в этом месяце со статусом "апартаменты"

**offers_in_sites_ratio** - доля офферов в ЖК в этом районе в этом месяце

**studio_ratio** - доля объявлений о продаже студии среди всех объявлений в этом районе в этом месяце

**rooms_offered_*_ratio** - доля квартир указанной в * комнатности от всех квартир в продаже в этом районе в этом месяце

**unique_active_buildings_count** - количество домов, в которых есть квартиры в продаже в этом районе в этом месяце

**unique_active_building_series_count** - количество уникальных серий домов, в которых есть квартиры в продаже в этом районе в этом месяце

**total_area_avg** - средний метраж продаваемой квартиры в этом районе в этом месяце

**offer_count** - число объявлений о продаже квартир в этом районе в этом месяце

**avg_price_sqm** - средняя цена за квадратный метр в этом районе в этом месяце

### housebase.csv

Описание полей в датасете про базу домов housebase.csv:

**city_quadkey** - полигон, его координаты можно его узнать из скрипта quadkey.py

**building_id** - id дома

**building_series_id** - id серии дома

**unified_address** - адрес дома

**latitude, longitude** - координаты дома

**flats_count** - количество квартир в доме

**has_elevator** - наличие лифта в доме

**ceiling_height** - высота потолков в доме

**build_year** - год постройки дома

**expect_demolition** - дом входит в программу реновации и ожидает сноса

**art_cnt** - количество организаций в сфере искусства (музеи, галереи и т.п.) в этом районе city_quadkey

**beauty_cnt** - количество салонов красоты в этом районе city_quadkey

**cafe_restaurant_eating_out_cnt** - количество точек общественного питания в этом районе city_quadkey

**chain_cnt** - количество сетевых организаций в этом районе city_quadkey

**entertainment_cnt** - количество организаций в сфере развлечений в этом районе city_quadkey

**groceries_and_everyday_items_cnt** - количество продуктовых магазинов в этом районе city_quadkey

**healthcare_cnt** - количество организаций в сфере здравоохранения в этом районе city_quadkey

**laundry_and_repair_services_cnt** - количество прачечных и ремонтных мастерских в этом районе city_quadkey

**shopping_cnt** - количество магазинов в этом районе city_quadkey

**sport_cnt** - количество спортивных организаций в этом районе city_quadkey

In [5]:
train.head()

,month,city_quadkey,apartment_ratio,offers_in_sites_ratio,studio_ratio,rooms_offered_1_ratio,rooms_offered_2_ratio,rooms_offered_3_ratio,rooms_offered_4_more_ratio,total_area_avg,unique_active_buildings_count,unique_active_building_series_count,offer_count,avg_price_sqm
0,2017-01-01,120310101302200,0.0,0.0,0.0,0.111111,0.666667,0.222222,0.000000,50.833333,4,3,9,158925.802951
1,2017-01-01,120310101120111,0.0,0.0,0.0,0.000000,0.571429,0.428571,0.000000,62.028571,4,4,7,163885.224330
2,2017-01-01,120310101321232,0.0,0.0,0.0,0.368421,0.350877,0.245614,0.035088,50.638597,26,9,57,150508.029742
3,2017-01-01,120310112010101,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.000000,36.250000,3,1,4,65206.228516
4,2017-01-01,120132323332000,0.0,0.0,0.0,0.357143,0.357143,0.285714,0.000000,50.264286,9,2,14,75161.611049


In [6]:
test.head()

,month,city_quadkey,apartment_ratio,offers_in_sites_ratio,studio_ratio,rooms_offered_1_ratio,rooms_offered_2_ratio,rooms_offered_3_ratio,rooms_offered_4_more_ratio,total_area_avg,unique_active_buildings_count,unique_active_building_series_count,offer_count,avg_price_sqm
0,2019-01-01,120310113123120,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,32.000000,1,1,1,34375.000000
1,2019-01-01,120310101222111,0.0,0.0,0.000000,0.333333,0.428571,0.238095,0.000000,46.385238,12,5,21,130991.212426
2,2019-01-01,120132321211130,0.0,0.0,0.071429,0.357143,0.500000,0.071429,0.071429,39.857143,8,1,14,45630.522042
3,2019-01-01,120310101310330,0.0,0.0,0.000000,0.000000,0.666667,0.333333,0.000000,51.033333,6,4,6,137194.388021
4,2019-01-01,120310101313200,0.0,0.0,0.000000,0.181818,0.469697,0.287879,0.060606,62.851515,31,4,66,106538.841205


In [7]:
housebase.head()

,city_quadkey,building_id,building_series_id,building_type,unified_address,latitude,longitude,flats_count,has_elevator,ceiling_height,build_year,expect_demolition,art_cnt,beauty_cnt,cafe_restaurant_eating_out_cnt,chain_cnt,entertainment_cnt,groceries_and_everyday_items_cnt,healthcare_cnt,laundry_and_repair_services_cnt,shopping_cnt,sport_cnt
0,120310101302011,3031030817531317628,0,BRICK,"Россия, Москва, 1-й Автозаводский проезд, 3",55.705219,37.656994,0,1,NaN,1937.0,0,7.0,36.0,24.0,127.0,18.0,32.0,16.0,38.0,45.0,13.0
1,120310101302011,3031030817531317630,0,BRICK,"Россия, Москва, 1-й Автозаводский проезд, 5",55.704845,37.657295,0,0,NaN,1962.0,0,7.0,36.0,24.0,127.0,18.0,32.0,16.0,38.0,45.0,13.0
2,120310101302011,1764947998512909470,0,NaN,"Россия, Москва, 1-й Автозаводский проезд, 7/1",55.704697,37.658203,0,0,NaN,NaN,0,7.0,36.0,24.0,127.0,18.0,32.0,16.0,38.0,45.0,13.0
3,120310101031222,2614890160181791580,1564812,BRICK,"Россия, Москва, 1-й Амбулаторный проезд, 2",55.811642,37.535599,176,1,2.65,1967.0,0,4.0,6.0,11.0,60.0,16.0,25.0,14.0,25.0,23.0,5.0
4,120310101031222,4152249910202725078,1564812,BRICK,"Россия, Москва, 1-й Амбулаторный проезд, 5к1",55.810810,37.534649,60,0,NaN,1959.0,0,4.0,6.0,11.0,60.0,16.0,25.0,14.0,25.0,23.0,5.0


In [8]:
f'Количество объектов в train {len(train)}, в test {len(test)}, количество записей в housebase {len(housebase)}.'

'Количество объектов в train 123491, в test 41280, количество записей в housebase 112203.'

## EDA

Посмотрим есть ли пропуски в данных:

In [9]:
train.isna().sum()

month                                  0
city_quadkey                           0
apartment_ratio                        0
offers_in_sites_ratio                  0
studio_ratio                           0
rooms_offered_1_ratio                  0
rooms_offered_2_ratio                  0
rooms_offered_3_ratio                  0
rooms_offered_4_more_ratio             0
total_area_avg                         0
unique_active_buildings_count          0
unique_active_building_series_count    0
offer_count                            0
avg_price_sqm                          0
dtype: int64

In [10]:
test.isna().sum()

month                                  0
city_quadkey                           0
apartment_ratio                        0
offers_in_sites_ratio                  0
studio_ratio                           0
rooms_offered_1_ratio                  0
rooms_offered_2_ratio                  0
rooms_offered_3_ratio                  0
rooms_offered_4_more_ratio             0
total_area_avg                         0
unique_active_buildings_count          0
unique_active_building_series_count    0
offer_count                            0
avg_price_sqm                          0
dtype: int64

In [11]:
housebase.isna().sum()

city_quadkey                            0
building_id                             0
building_series_id                      0
building_type                       22454
unified_address                         0
latitude                                0
longitude                               0
flats_count                             0
has_elevator                            0
ceiling_height                      65357
build_year                          29285
expect_demolition                       0
art_cnt                             27401
beauty_cnt                          23037
cafe_restaurant_eating_out_cnt      28397
chain_cnt                            6905
entertainment_cnt                   12357
groceries_and_everyday_items_cnt     8726
healthcare_cnt                      19610
laundry_and_repair_services_cnt     13139
shopping_cnt                        19639
sport_cnt                           19309
dtype: int64

Посмотрим на типы колонок:

In [12]:
train.dtypes

month                                   object
city_quadkey                             int64
apartment_ratio                        float64
offers_in_sites_ratio                  float64
studio_ratio                           float64
rooms_offered_1_ratio                  float64
rooms_offered_2_ratio                  float64
rooms_offered_3_ratio                  float64
rooms_offered_4_more_ratio             float64
total_area_avg                         float64
unique_active_buildings_count            int64
unique_active_building_series_count      int64
offer_count                              int64
avg_price_sqm                          float64
dtype: object

In [13]:
test.dtypes

month                                   object
city_quadkey                             int64
apartment_ratio                        float64
offers_in_sites_ratio                  float64
studio_ratio                           float64
rooms_offered_1_ratio                  float64
rooms_offered_2_ratio                  float64
rooms_offered_3_ratio                  float64
rooms_offered_4_more_ratio             float64
total_area_avg                         float64
unique_active_buildings_count            int64
unique_active_building_series_count      int64
offer_count                              int64
avg_price_sqm                          float64
dtype: object

In [14]:
housebase.dtypes

city_quadkey                          int64
building_id                           int64
building_series_id                    int64
building_type                        object
unified_address                      object
latitude                            float64
longitude                           float64
flats_count                           int64
has_elevator                          int64
ceiling_height                      float64
build_year                          float64
expect_demolition                     int64
art_cnt                             float64
beauty_cnt                          float64
cafe_restaurant_eating_out_cnt      float64
chain_cnt                           float64
entertainment_cnt                   float64
groceries_and_everyday_items_cnt    float64
healthcare_cnt                      float64
laundry_and_repair_services_cnt     float64
shopping_cnt                        float64
sport_cnt                           float64
dtype: object

Разделим признаки на несколько групп:

**-1:** `avg_price_sqm` (средняя цена, наша целевая переменная)

**0:** `city_quadkey`, `month` (район и месяц, для чего мы предсказываем)

**1:** `apartment_ratio`, `offers_in_sites_ratio`, `studio_ratio`, `rooms_offered_1_ratio`, `rooms_offered_2_ratio`, `rooms_offered_3_ratio`, `rooms_offered_4_more_ratio`, `total_area_avg` (описание объвлений на районе в этом месяце тип 1, из train)

**2:** `unique_active_buildings_count`, `unique_active_building_series_count`, `offer_count` (описание объявлений на районе в этом месяце тип 2, из train)

**3:** `beauty_cnt`, `shopping_cnt`, `cafe_restaurant_eating_out_cnt`, `entertainment_cnt`, `sport_cnt`, `chain_cnt`, `groceries_and_everyday_items_cnt`, `art_cnt`, `healthcare_cnt`, `laundry_and_repair_services_cnt` (описание района, из housebase)

**4:** `building_id`, `building_series_id`, `building_type`, `unified_address`, `flats_count`, `expect_demolition`, `ceiling_height`, `has_elevator`, `build_year` + `latitude`, `longitude` (описание дома, из housebase)

### Группа -1

Постройте график распределения целевой переменной (`avg_price_sqm`). Здесь и далее лучше использовать функцию `distplot` из библиотеки `seaborn`.

In [15]:
### YOUR CODE HERE
# sns.distplot(train['avg_price_sqm'])

### Группа 0

- Выведите все уникальные значения месяцев в датасете, пересекаются ли они в обучающей и тестовой выборке?
- Постройте график распределения количества объявлений в зависимости от месяца для обучающей выборки.
- Посчитайте сколько уникальных районов представлено в датасете, в обучающей и тестовой выборке. Есть ли в тестовой выборке новые районы?

In [16]:
### YOUR CODE HERE
# city_quadkey, mont
# test['month'].unique()
# train['month'].unique()
# train.plot(x='month',y='offer_count')

In [17]:
# print(train['city_quadkey'].nunique())
# print(test['city_quadkey'].nunique())
# np.in1d(train['city_quadkey'].nunique(),test['city_quadkey'].nunique())

Запишите в переменные (`train_nunique_month` и `train_nunique_quadkey`) количество уникальных значений в обучающей выборке для признаков `month` и `city_quadkey`.

In [18]:
# your code here
train_nunique_month = train['month'].nunique()
train_nunique_quadkey = train['city_quadkey'].nunique()

In [19]:
### GRADING, RUN TO TEST

train_nunique_month
train_nunique_quadkey

print('... TESTING ...')


... TESTING ...


### Группа 1

Для каждого признака постройте график распределения. Есть ли какие-то неинформативные признаки, которые можно сразу выбросить?

In [20]:
### YOUR CODE HERE
# columns = ['apartment_ratio', 'offers_in_sites_ratio', 'studio_ratio', 'rooms_offered_1_ratio',
#       'rooms_offered_2_ratio', 'rooms_offered_3_ratio', 'rooms_offered_4_more_ratio', 'total_area_avg']
# for column in columns:
#     sns.histplot(train[column])
#     plt.show()

Запишите в переменную `unwanted_cols` список с признаками, которые можно будет удалить.

In [21]:
# your code here
unwanted_cols = ['offers_in_sites_ratio']

In [22]:
### GRADING

unwanted_cols

print('... TESTING ...')


... TESTING ...


### Группа 2

Для каждого признака постройте график распределения. Есть ли какие-то неинформативные признаки, которые можно сразу выбросить?

In [23]:
### YOUR CODE HERE
# columns = ['unique_active_buildings_count', 'unique_active_building_series_count', 'offer_count']
# for column in columns:
#     sns.histplot(train[column])
#     plt.show()

### Группа 3

Для каждого признака постройте график распределения. 

Так как в этой группе признаки содержат информацию про количество тех или иных объектов на районе, физический смысл отсутствия информации - на районе нет таких объектов. В дальнейшем мы воспользуемся этой идеей для заполнения пропусков.

In [24]:
### YOUR CODE HERE
# columns = ['beauty_cnt', 'shopping_cnt', 'cafe_restaurant_eating_out_cnt',
#            'entertainment_cnt', 'sport_cnt', 'chain_cnt',
#            'groceries_and_everyday_items_cnt', 'art_cnt',
#            'healthcare_cnt', 'laundry_and_repair_services_cnt']
# for column in columns:
#     sns.histplot(housebase[column])
#     plt.show()

### Группа 4

- Постройте графики распределения для `building_series_id` и `building_type`.
- Постройте графики распределения для `flats_count`, `ceiling_height` и `build_year`.
- Постройте графики распределения для `has_elevator` и `expect_demolition`.

In [25]:
### YOUR CODE HERE
# columns = ['building_id', 'building_series_id', 'building_type',
#            'unified_address', 'flats_count', 'expect_demolition',
#            'ceiling_height', 'has_elevator', 'build_year', 'latitude',
#            'longitude']
# sns.displot(housebase['building_series_id'])
# sns.catplot(data=housebase,x='building_type',kind='count',height=6, aspect=2)
# sns.displot(housebase['flats_count'])
# sns.displot(housebase['ceiling_height'])
# sns.displot(housebase['build_year'])
# sns.displot(housebase['has_elevator'])
# sns.displot(housebase['expect_demolition'])
# plt.show()

### Деление на категориальные и вещественные

Разделите признаки на категориальные и вещественные, запишите их в переменные `cat_features` и `real_features`. Какие признаки вы не будете использовать?

In [26]:
# your code here
cat_features = ['building_type',
                'expect_demolition','has_elevator','is_in_Moscow']

real_features = ['apartment_ratio', 'studio_ratio',
                'rooms_offered_1_ratio', 'rooms_offered_2_ratio', 
                'rooms_offered_3_ratio', 'rooms_offered_4_more_ratio', 'total_area_avg',
                'unique_active_buildings_count',
                'unique_active_building_series_count', 'offer_count',
                'beauty_cnt', 'shopping_cnt', 'cafe_restaurant_eating_out_cnt',
                'entertainment_cnt', 'sport_cnt',
                'chain_cnt', 'groceries_and_everyday_items_cnt',
                'art_cnt', 'healthcare_cnt',
                'laundry_and_repair_services_cnt','build_year',
                'ceiling_height',
                'flats_count','center_dist']

## Генерация признаков

Сгенерируем несколько новых признаков, разделим их на несколько групп:

- Отсутствие информации про район/дом
- Находится ли дом в Москве
- Расстояние до центра города
- Находится ли дом внутри колец
- Время года

### Отсутствие информации про район/дом

Вспомните признаки из группы 3 и группы 4, для каждого признака сделайте бинарный признак, который будет обозначать, что информация отсутствует, назовите их `col_name + '_isna'`. Создавать новые признаки стоит в `housebase`. Куда стоит отнести новые признаки? 

Заполните пропуски: для категориальных новым значением `UNKNOWN`, для вещественных нулём.

In [27]:
columns_3 = ['beauty_cnt', 'shopping_cnt', 'cafe_restaurant_eating_out_cnt',
           'entertainment_cnt', 'sport_cnt', 'chain_cnt',
           'groceries_and_everyday_items_cnt', 'art_cnt',
           'healthcare_cnt', 'laundry_and_repair_services_cnt']
columns_4 = ['building_id', 'building_series_id', 'building_type',
           'unified_address', 'flats_count', 'expect_demolition',
           'ceiling_height', 'has_elevator', 'build_year', 'latitude',
           'longitude']
for column in columns_3 + columns_4:
    housebase[column+'_isna']=housebase[column].isna().astype(int)

In [28]:
values = {}
for feature in cat_features:
    values[feature]='UNKNOWN'
for feature in real_features:
    values[feature]=0
values

{'building_type': 'UNKNOWN',
 'expect_demolition': 'UNKNOWN',
 'has_elevator': 'UNKNOWN',
 'is_in_Moscow': 'UNKNOWN',
 'apartment_ratio': 0,
 'studio_ratio': 0,
 'rooms_offered_1_ratio': 0,
 'rooms_offered_2_ratio': 0,
 'rooms_offered_3_ratio': 0,
 'rooms_offered_4_more_ratio': 0,
 'total_area_avg': 0,
 'unique_active_buildings_count': 0,
 'unique_active_building_series_count': 0,
 'offer_count': 0,
 'beauty_cnt': 0,
 'shopping_cnt': 0,
 'cafe_restaurant_eating_out_cnt': 0,
 'entertainment_cnt': 0,
 'sport_cnt': 0,
 'chain_cnt': 0,
 'groceries_and_everyday_items_cnt': 0,
 'art_cnt': 0,
 'healthcare_cnt': 0,
 'laundry_and_repair_services_cnt': 0,
 'build_year': 0,
 'ceiling_height': 0,
 'flats_count': 0,
 'center_dist': 0}

In [29]:
housebase.fillna(values,inplace=True)
test.fillna(values,inplace=True)
train.fillna(values,inplace=True)

In [30]:
### GRADING

for col in ['art_cnt', 'beauty_cnt', 'cafe_restaurant_eating_out_cnt', 'chain_cnt', 'entertainment_cnt', 'groceries_and_everyday_items_cnt', 'healthcare_cnt', 'laundry_and_repair_services_cnt', 'shopping_cnt', 'sport_cnt']:
    assert col + '_isna' in housebase.columns

print('... TESTING ...')

... TESTING ...


### Находится ли дом в Москве

Сделайте новый бинарный признак, который будет обозначать находится ли дом в Москве, назовите его `is_in_Moscow`. Для этого стоит анализировать уже имеющийся признак `unified_address`.

In [31]:
# your code here
housebase['is_in_Moscow'] = housebase['unified_address'].apply(lambda x: x.split(',')).apply(lambda x: 1 if ' Москва' in x or 'Москва' in x else 0)

In [32]:
### GRADING

assert 'is_in_Moscow' in housebase.columns

print('... TESTING ...')


... TESTING ...


### Расстояние до центра города

Создайте новый вещественный признак, который будет равен расстоянию от дома до Кремля, назовите его `center_dist`. Координаты Кремля: (55.7528, 37.6178).

In [33]:
!pip install haversine
from haversine import haversine

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [34]:
# your code hereLatitude and longitude
msk = [55.7528, 37.6178]
housebase['center_dist'] = housebase[['latitude','longitude']].apply(lambda x:haversine(x,msk,unit='km'),axis=1)
housebase['center_dist']

0          5.832208
1          5.877863
2          5.916958
3          8.320408
4          8.285069
            ...    
112198    49.404777
112199    76.145836
112200     2.068485
112201     8.981164
112202     3.977095
Name: center_dist, Length: 112203, dtype: float64

In [35]:
### GRADING

assert 'center_dist' in housebase.columns

print('... TESTING ...')


... TESTING ...


### Находится ли дом внутри колец

Создайте три новых бинарных признака - находится ли дом внутри Бульварного, Садового или Третьего Транспортного Кольца. Для этого стоит сравнить уже полученное расстояние до Кремля с 1.5, 3 и 6 километрами, соответственно.

In [36]:
### YOUR CODE HERE
housebase['inside_bulvar'] = housebase['center_dist'].apply(lambda x: x<1.5).astype(int)
housebase['inside_cad'] = housebase['center_dist'].apply(lambda x: x<3).astype(int)
housebase['inside_ttk'] = housebase['center_dist'].apply(lambda x: x<6).astype(int)

### Время года

Добавьте новый категориальный признак, который будет равен времени года.

In [37]:
### YOUR CODE HERE
seasons = {
    '12' : 'winter',
    '01' : 'winter',
    '02' : 'winter',
    '03' : 'spring',
    '04' : 'spring',
    '05' : 'spring',
    '06' : 'summer',
    '07' : 'summer',
    '08' : 'summer',
    '09' : 'fall',
    '10' : 'fall',
    '11' : 'fall',
}
train['season'] = train['month'].apply(lambda x: seasons[x.split('-')[1]])
test['season'] = test['month'].apply(lambda x: seasons[x.split('-')[1]])

## Подготовка данных для обучения

Теперь нам надо объединить обе таблички, чтобы добавить информацию про дома для каждого района. Перед тем как объединять таблички нам нужно будет создать для каждого района образ "среднего" дома по району.

Для этого мы для каждого категориального признака возьмем моду, для каждого вещественного медиану.

Произведите вышеописанные манипуляции с housebase.

In [38]:
real_housebase = np.intersect1d(real_features, housebase.columns)
cat_housebase = np.intersect1d(cat_features, housebase.columns)
def combine_housebase(quadkeys_):
    groups = housebase.loc[housebase['city_quadkey'].isin(quadkeys_)].groupby('city_quadkey')

    aggs = {}
    for feature in cat_housebase:
        aggs[feature]=lambda x: x.mode()[0]
    for feature in real_housebase:
        aggs[feature]=lambda x: x.median()
    
    return groups.agg(aggs).reset_index(drop=False)

In [39]:
from multiprocessing import Pool, cpu_count
from itertools import chain

try:
    with Pool(cpu_count()) as pool:
        results = pool.map(combine_housebase, np.array_split(housebase['city_quadkey'].unique(), cpu_count()))
except KeyboardInterrupt:
    pass

housebase = pd.concat(results)

In [40]:
### GRADING

assert len(housebase) == housebase['city_quadkey'].nunique()

print('... TESTING ...')


... TESTING ...


In [41]:
train_data = train.join(housebase.set_index('city_quadkey'), on='city_quadkey', how='left')
test_data = test.join(housebase.set_index('city_quadkey'), on='city_quadkey', how='left')

Добавим еще один вещественный признак - возраст дома.

In [42]:
train_data['build_year'] = train_data['month'].str.split('-').map(lambda x: x[0]).astype(int) - train_data['build_year']
test_data['build_year'] = test_data['month'].str.split('-').map(lambda x: x[0]).astype(int) - test_data['build_year']

In [43]:
train_data.columns

Index(['month', 'city_quadkey', 'apartment_ratio', 'offers_in_sites_ratio',
       'studio_ratio', 'rooms_offered_1_ratio', 'rooms_offered_2_ratio',
       'rooms_offered_3_ratio', 'rooms_offered_4_more_ratio', 'total_area_avg',
       'unique_active_buildings_count', 'unique_active_building_series_count',
       'offer_count', 'avg_price_sqm', 'season', 'building_type',
       'expect_demolition', 'has_elevator', 'is_in_Moscow', 'art_cnt',
       'beauty_cnt', 'build_year', 'cafe_restaurant_eating_out_cnt',
       'ceiling_height', 'center_dist', 'chain_cnt', 'entertainment_cnt',
       'flats_count', 'groceries_and_everyday_items_cnt', 'healthcare_cnt',
       'laundry_and_repair_services_cnt', 'shopping_cnt', 'sport_cnt'],
      dtype='object')

In [44]:
train_data.drop(['month', 'city_quadkey','offers_in_sites_ratio'],inplace=True,axis=1)
test_data.drop(['month', 'city_quadkey','offers_in_sites_ratio'],inplace=True,axis=1)

Разделим данные на признаки и целевую переменную.

In [45]:
x_train, y_train = train_data.drop(['avg_price_sqm'], axis=1), train_data[['avg_price_sqm']]
x_test, y_test = test_data.drop(['avg_price_sqm'], axis=1), test_data[['avg_price_sqm']]

В качестве бейзлайна возьмем значение нашей метрики (MAPE) для медианы обучающей выборки:

In [46]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [47]:
y_dummy = y_train.median()

f'MAPE baseline: TRAIN {mape(y_train, y_dummy):.2f}, TEST {mape(y_test, y_dummy):.2f}'

'MAPE baseline: TRAIN 50.77, TEST 54.41'

## Перебор гиперпараметров

Заполните пропуски в функции для поиска лучших гиперпараметров для модели.

In [48]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [49]:
from sklearn.metrics import make_scorer

scorer = make_scorer(mape, greater_is_better=False)


def hyperopt(estimator, params, cat=None, real=None):
    if cat is None:
        cat = cat_features
    
    if real is None:
        real = real_features

    column_transformer = ColumnTransformer(
        transformers=[
            ('first',OneHotEncoder(handle_unknown = 'ignore'),cat),
            ('second',StandardScaler(),real)
            
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor' ,column_transformer),
            ('estimator' , estimator)
            
        ]
    )
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        scoring=scorer,
        cv=3,
        verbose=10,
        n_jobs=1
    )
    
    grid.fit(x_train, y_train)
    
    # write best params to `best_params`
    # your code here
    best_params = grid.best_params_
    
    score_train = mape(y_train, grid.predict(x_train))
    score_test = mape(y_test, grid.predict(x_test))
    
    return score_train, score_test, best_params

In [50]:
### GRADING

print('... TESTING ...')


... TESTING ...


## Сравнение моделей

Подберите для каждой из следующих моделей лучшие гиперпараметры с помощью функции, которую вы написали выше.

- Линейная модель с l2 регуляризацией
- Решающее дерево
- Случайный лес
- Градиентный бустинг
- Метод ближайших соседей

### Линейная модель с l2 регуляризацией

Попробуйте исследовать влияние `alpha` и `max_iter` на результат.

In [51]:
from sklearn.linear_model import Ridge
params = {
    'estimator__max_iter': [300,400,500,600,700],
    'estimator__alpha':[4,3,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]
    
}
hyperopt(Ridge(),params)

### YOUR CODE HERE


Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3; 1/60] START estimator__alpha=4, estimator__max_iter=300................
[CV 1/3; 1/60] END estimator__alpha=4, estimator__max_iter=300;, score=-26.892 total time=   0.2s
[CV 2/3; 1/60] START estimator__alpha=4, estimator__max_iter=300................
[CV 2/3; 1/60] END estimator__alpha=4, estimator__max_iter=300;, score=-27.359 total time=   0.2s
[CV 3/3; 1/60] START estimator__alpha=4, estimator__max_iter=300................
[CV 3/3; 1/60] END estimator__alpha=4, estimator__max_iter=300;, score=-26.289 total time=   0.1s
[CV 1/3; 2/60] START estimator__alpha=4, estimator__max_iter=400................
[CV 1/3; 2/60] END estimator__alpha=4, estimator__max_iter=400;, score=-26.892 total time=   0.2s
[CV 2/3; 2/60] START estimator__alpha=4, estimator__max_iter=400................
[CV 2/3; 2/60] END estimator__alpha=4, estimator__max_iter=400;, score=-27.359 total time=   0.2s
[CV 3/3; 2/60] START estimator__alpha=4, es

(26.747692736149386,
 25.035701264860034,
 {'estimator__alpha': 4, 'estimator__max_iter': 300})

Запишите в переменную `best_params_ridge` ваши лучшие гиперпараметры для `Ridge`.

In [52]:
# your code here
best_params_ridge = {'alpha':4,
                    'max_iter' :300}

In [53]:
### GRADING

best_params_ridge

print('... TESTING ...')


... TESTING ...


### Решающее дерево

Попробуйте исследовать влияние `max_depth`, `min_samples_split` и `max_features` на результат.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
params = {
    'estimator__max_depth': list(range(1,6)),
    'estimator__min_samples_split':[2,5,10],
    'estimator__max_features':[2,5,10]
    
}
hyperopt(DecisionTreeRegressor(),params)

### YOUR CODE HERE


Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV 1/3; 1/45] START estimator__max_depth=1, estimator__max_features=2, estimator__min_samples_split=2


Запишите в переменную `best_params_dt` ваши лучшие гиперпараметры для `DecisionTreeRegressor`.

In [ ]:
# your code here
best_params_dt = {
        'max_depth': ,
        'min_samples_split': ,
        'max_features':
}

In [ ]:
### GRADING

best_params_dt

print('... TESTING ...')


### Случайный лес

Попробуйте исследовать влияние `n_estimators`, `max_depth`, `min_samples_split` и `max_features` на результат.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
params = {
    'estimator__n_estimators': list(range(1,100,20)),
    'estimator__max_depth': list(range(1,15)),
    'estimator__min_samples_split':list(range(1,20)),
    'estimator__max_features':list(range(1,20))
    
}
hyperopt(RandomForestRegressor(),params)

### YOUR CODE HERE


Запишите в переменную `best_params_rf` ваши лучшие гиперпараметры для `RandomForestRegressor`.

In [ ]:
# your code here
best_params_rf = {
        'estimator__n_estimators': ,
        'max_depth': ,
        'min_samples_split': ,
        'max_features':
}

In [ ]:
### GRADING

best_params_rf

print('... TESTING ...')


### Градиентный бустинг

Попробуйте исследовать влияние `n_estimators`, `learning_rate`, `subsample` и `min_samples_split` на результат.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

params = {
    'estimator__n_estimators': list(range(1,100,20)),
    'estimator__learning_rate': [0.1,0.2,0.5,1],
    'estimator__subsample':[0.5,1],
    'estimator__min_samples_split':list(range(1,7))
    
}
hyperopt(GradientBoostingRegressor(),params)
### YOUR CODE HERE


Запишите в переменную `best_params_gb` ваши лучшие гиперпараметры для `GradientBoostingRegressor`.

In [ ]:
# your code here
best_params_gb = {
    'n_estimators': ,
    'learning_rate': ,
    'subsample': ,
    'min_samples_split':
    
}

In [ ]:
### GRADING

best_params_gb

print('... TESTING ...')


### Метод ближайших соседей

Попробуйте исследовать влияние `n_neighbors`, `algorithm` и `leaf_size` на результат.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
params = {
    'estimator__n_neighbors': [1,20,30,50,100],
    'estimator__algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'estimator__leaf_size':[1,10,20,30]
    
}
hyperopt(KNeighborsRegressor(),params)

### YOUR CODE HERE


Запишите в переменную `best_params_knb` ваши лучшие гиперпараметры для `KNeighborsRegressor`.

In [ ]:
# your code here
best_params_knb = {
    'n_neighbors': ,
    'algorithm': ,
    'leaf_size':
    
}

In [ ]:
### GRADING

best_params_knb

print('... TESTING ...')


## Сравнение моделей

Сравните модели с лучшими гиперпараметрами между собой по

- Значению метрики на обучающей выборке
- Значению метрики на тестовой выборке
- Времени на обучение на обучающей выборке

In [ ]:
### YOUR CODE HERE
